## Ceneo Scrapper

## Struktura pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation|recommendation|
|gwiazdki|span.user-post__score-count|stars|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature__title--positives ~ div.review-feature items|pros|
|lista wad|div.review-feature__title--negatives ~ div.review-feature items|cons|
|dla ilu przydatna|button.vote-yes > span|helpful|
|dla ilu nieprzydatna|button.vote-no > span|unhelpful|
|data wystawienia|span.user-post__published > time:nth-child(1)("datetime")|publish date|
|data zakupu|span.user-post__published > time:nth-child(2)("datetime")|purchase date|

## Zainstalowanie bibliotek

In [18]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Wysłanie zapytania do serweru

In [ ]:
product_id = '138331381'
url = f"https://ceneo.pl/{product_id}#tab=rewievs"
response = requests.get(url)

## Pobranie opinii z kodu HTML strony

In [13]:
page_dom = BeautifulSoup(response.text, 'html.parser')
opinions = page_dom.select("div.js_product-review")

In [14]:
all_opinions = []
for opinion in opinions:
    single_opinion = {
        "opinion_id": opinion["data-entry-id"],
        "author" : opinion.select_one("span.user-post__author-name").text.strip(),
        "recommendation" : opinion.select_one("span.user-post__author-recomendation").text.strip(),
        "stars" : opinion.select_one("span.user-post__score-count").text.strip(),
        "contents" : opinion.select_one("div.user-post__text").text.strip(),
        "pros" : [p.text.strip()  for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
        "cons" : [p.text.strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
        "helpful" : opinion.select_one("button.vote-yes > span").text.strip(),
        "unhelpful" : opinion.select_one("button.vote-no > span").text.strip(),
        "publish_date" : opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"],
        "purchase_date" : opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
    }
    all_opinions.append(single_opinion)


In [19]:
if not os.path.exists("opinions"):
    os.makedirs("opinions")
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf: #jf = json_file
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

In [20]:
next_page = page_dom.select_one("a.pagination__next")["href"].strip()
print(next_page)